## Members
- Cao Thanh Khiết   19120544
- Ninh Duy Huy      
- Lê Minh Hữu
- xxx
- xxx
# Task of each member
- Cao Thanh Khiết
    - Use SoundClound API to get data and write down them into csv file
- Ninh Duy Huy
    -
- Lê Minh Hữu
    - xxx
- xxx
    - xxx
- xxx
    - xxx

In [6]:
# More information in README.md file
import requests
import pandas as pd

In [7]:
api = "https://api-v2.soundcloud.com"
client_id = "6gsNBd4mJwXr0LxTBh8VKBOrViK6Aj56"
client_id_query = "?client_id={}".format(client_id)

In [8]:
# "top50_url.txt" contains url of top lists ( system-playlists of top 50 tracks)
# This cell Convert url_lists from txt file to List python
url_entrypoint = "url_entry/top50_url.txt"
# url_entrypoint = "url_entry/top50_url_max.txt"    # uncomment here to get more records, but it take much longer time (nearly 3 hours)
sys_playlists_url = []

with open(url_entrypoint) as file:
    lines = [ sys_playlists_url.append(line.rstrip()) for line in file]

In [9]:
# Sending request for each system-playlists to get track_id of top system-playlists
track_ids = []  # holding id of track

for sys_url in sys_playlists_url:
    try:
        resolve_api = api + "/resolve" + client_id_query + "&url={}".format(sys_url)
        res = requests.get(resolve_api)
        for track in res.json()['tracks']:
            track_ids.append(track['id'])
    except KeyError:
        pass

In [ ]:
# From that track_id, we send request for each tracks to get user_id of each track
user_ids = []   # containing id of user

for track in track_ids:
    try:
        track_api = api + "/tracks/{}".format(track) + client_id_query
        res = requests.get(track_api)
        user_ids.append(res.json()['user']['id'])
    except KeyError:
        pass

In [ ]:
users = []  # containing detail information of each user

for user_id in user_ids:
    user_api = api + "/users/{}".format(user_id) + client_id_query
    res = requests.get(user_api)
    users.append(res.json())

In [12]:
# Define a schema for User Dataframe and converting to CSV file
users_table = pd.DataFrame({
    'id': pd.Series([u['id'] if u['id'] else 'None' for u in [user for user in users]]),
    'username': pd.Series([u['username'] if u['username'] else 'None' for u in [user for user in users]]),
    'first_name': pd.Series([u['first_name'] if u['first_name'] else 'None' for u in [user for user in users]]),
    'last_name': pd.Series([u['last_name'] if u['last_name'] else 'None' for u in [user for user in users]]),
    'full_name': pd.Series([u['full_name'] if u['full_name'] else 'None' for u in [user for user in users]]),
    'description': pd.Series([u['description'] if u['description'] else 'None' for u in [user for user in users]]),
    'verified': pd.Series([u['verified'] if u['verified'] else 'None' for u in [user for user in users]]),
    'city': pd.Series([u['city'] if u['city'] else 'None' for u in [user for user in users]]),
    'country_code': pd.Series([u['country_code'] if u['country_code'] else 'None' for u in [user for user in users]]),
    'followers_count': pd.Series([u['followers_count'] if u['followers_count'] else 'None' for u in [user for user in users]]),
    'followings_count': pd.Series([u['followings_count'] if u['followings_count'] else 'None' for u in [user for user in users]]),
    'comments_count': pd.Series([u['comments_count'] if u['comments_count'] else 'None' for u in [user for user in users]]),
    'track_count': pd.Series([u['track_count'] if u['track_count'] else 'None' for u in [user for user in users]]),
    'avatar_url': pd.Series([u['avatar_url'] if u['avatar_url'] else 'None' for u in [user for user in users]]),
    'groups_count': pd.Series([u['groups_count'] if u['groups_count'] else 'None' for u in [user for user in users]]),
    'playlist_count': pd.Series([u['playlist_count'] if u['playlist_count'] else 'None' for u in [user for user in users]]),
    'reposts_count': pd.Series([u['reposts_count'] if u['reposts_count'] else 'None' for u in [user for user in users]]),
    'last_modified': pd.Series([u['last_modified'] if u['last_modified'] else 'None' for u in [user for user in users]]),
    'created_at': pd.Series([u['created_at'] if u['created_at'] else 'None' for u in [user for user in users]])
})

users_table.to_csv('Api_data/users.csv')

In [ ]:
## This cell parse playlist_id and track_id, writing it into playlists.csv file
playlists = [] # Containing List of Dict, Dict includes playlist_id and track_ids property

for user_id in user_ids:
    user_api = api + "/users/{}/playlists".format(user_id) + client_id_query
    try:
        res = requests.get(user_api)
        collection = res.json()['collection']
        # loop
        for playlist in collection:
            playlist_id = playlist['id']
            track_id_string = ''
            for item in playlist['tracks']:
                track_id_string += str(item['id']) + ','
            track_id_string = track_id_string[:-1] if track_id_string != '' else 'None'
            playlists.append({
                'playlist_id': playlist_id,
                'track_ids': track_id_string
            })
    except KeyError:
        pass

playlists_table = pd.DataFrame(playlists)
playlists_table.to_csv('Api_data/playlists.csv')

In [ ]:
z

In [ ]:
# Get information of each track from playlists.csv file
tracks_detail = []
read_playlists = pd.read_csv('Api_data/playlists.csv')
# parse string from playlists.csv file to get track_id and send request to retrive detail information of each track

for index, row in read_playlists.iterrows():
    tracks = row['track_ids'].split(',')
    for track in tracks:
        
        if track == 'None':
            continue
        track_api = api + "/tracks/{}".format(track) + client_id_query
        res = requests.get(track_api)
        tracks_detail.append(res.json())

# Remove some unnessesary property of each track
key_remover = ['artwork_url', 'publisher_metadata', 'media', 'user', 'badges']

for key in key_remover:
    for dic in tracks_detail:
        dic.pop(key, None)

# convert infor track into csv file
track_df = pd.DataFrame(tracks_detail)
track_df.to_csv('Api_data/tracks.csv')
